In [1]:
# ==================================================================================================== Step 1: Import Libraries and Load Images
import cv2
import numpy as np

# Set the path to the images
img = [
    "img/no (1).jpg",
    "img/no (2).jpg",
    "img/no (3).jpg",
    "img/no (4).jpg",
    "img/no (5).jpg",
    "img/no (6).jpg",
    "img/no (7).jpg",
    "img/no (8).jpg",
    "img/no (9).jpg",
    "img/no (10).jpg",
    "img/gg (1).jpg",
    "img/gg (2).jpg",
    "img/gg (3).jpg",
    "img/gg (4).jpg",
    "img/m (1).jpg",
    "img/m (2).jpg",
    "img/m (3).jpg",
    "img/m (4).jpg",
    "img/m (5).jpg",
    "img/m (6).jpg",
]

# Shuffle the images
np.random.shuffle(img)

# Prepare the correct answers
correct_ans = []
for i in img:
    if "no" in i:
        correct_ans.append("No")
    else:
        correct_ans.append("Yes")

# Load the images in grayscale
df_img = []
for i in img:
    df_img.append(cv2.imread(i, cv2.IMREAD_GRAYSCALE))

In [2]:
# ==================================================================================================== Step 2: Preprocess the Images
# Crop image of unwanted padding
cropped_img = []
for img in df_img:
    # Threshold the image
    threshold = cv2.threshold(img.copy(), 40, 255, cv2.THRESH_OTSU)[1]
    # Find the contours
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get largest contour
    c = max(contours, key=cv2.contourArea)
    # Get extreme points of the contour
    extleft = tuple(c[c[:, :, 0].argmin()][0])
    extright = tuple(c[c[:, :, 0].argmax()][0])
    exttop = tuple(c[c[:, :, 1].argmin()][0])
    extbot = tuple(c[c[:, :, 1].argmax()][0])
    # Crop the image
    cropped_img.append(img[exttop[1] : extbot[1], extleft[0] : extright[0]])

# Show the cropped images
# for i in range(len(cropped_img)):
#     cv2.imshow("Cropped Image", cropped_img[i])
#     cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# ==================================================================================================== Step 3: Extract Features
# 1. Thresholding
threshold_img = []

thres_val = 100

for img in cropped_img:
    _, thres = cv2.threshold(img.copy(), thres_val, 255, cv2.THRESH_BINARY)
    threshold_img.append(thres)

# 2. Erode and Dilate
ed_img = []
k_size = 3
kernel = np.ones((k_size, k_size), np.uint8)
total_iterations = 1
e_iterations = 1
d_iterations = 1
for img in threshold_img:
    for i in range(total_iterations):
        erode = cv2.erode(img.copy(), kernel, iterations=e_iterations)
        dilate = cv2.dilate(erode, kernel, iterations=d_iterations)
    ed_img.append(dilate)

# 3. Finding Contours, Draw Contours, Extract features
contour_img = []
features = []

# Parameters

for img in ed_img:
    # Find the contours
    contours, _ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    new_canvas = np.zeros_like(img)
    cv2.drawContours(new_canvas, contours, -1, (255, 255, 255), 1)
    contour_img.append(contours)

    # Draw bounding box
    new_canvas = np.zeros_like(img)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        min_area = min_scale * img.shape[0] * img.shape[1]
        max_area = max_scale * img.shape[0] * img.shape[1]
        area = w * h
        bound_center = (x + w / 2, y + h / 2)
        low_x = box_pos_factor * img.shape[1]
        low_y = box_pos_factor * img.shape[0]
        up_x = img.shape[1] - box_pos_factor * img.shape[1]
        up_y = img.shape[0] - box_pos_factor * img.shape[0]

In [4]:
# ==================================================================================================== Step 4: Classify the Tumor